### This notebook is initial submission for the hackathon 

https://datahack.analyticsvidhya.com/contest/janatahack-cross-sell-prediction/

My public and private leaderboard scores are 0.8573981127 and 0.8625617278 respectively

In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

In [3]:
train_df.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


In [4]:
test_df.tail()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
127032,508142,Female,26,1,37.0,1,< 1 Year,No,30867.0,152.0,56
127033,508143,Female,38,1,28.0,0,1-2 Year,Yes,28700.0,122.0,165
127034,508144,Male,21,1,46.0,1,< 1 Year,No,29802.0,152.0,74
127035,508145,Male,71,1,28.0,1,1-2 Year,No,62875.0,26.0,265
127036,508146,Male,41,1,29.0,1,1-2 Year,No,27927.0,124.0,231


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381109 entries, 0 to 381108
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    381109 non-null  int64  
 1   Gender                381109 non-null  object 
 2   Age                   381109 non-null  int64  
 3   Driving_License       381109 non-null  int64  
 4   Region_Code           381109 non-null  float64
 5   Previously_Insured    381109 non-null  int64  
 6   Vehicle_Age           381109 non-null  object 
 7   Vehicle_Damage        381109 non-null  object 
 8   Annual_Premium        381109 non-null  float64
 9   Policy_Sales_Channel  381109 non-null  float64
 10  Vintage               381109 non-null  int64  
 11  Response              381109 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 34.9+ MB


In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127037 entries, 0 to 127036
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    127037 non-null  int64  
 1   Gender                127037 non-null  object 
 2   Age                   127037 non-null  int64  
 3   Driving_License       127037 non-null  int64  
 4   Region_Code           127037 non-null  float64
 5   Previously_Insured    127037 non-null  int64  
 6   Vehicle_Age           127037 non-null  object 
 7   Vehicle_Damage        127037 non-null  object 
 8   Annual_Premium        127037 non-null  float64
 9   Policy_Sales_Channel  127037 non-null  float64
 10  Vintage               127037 non-null  int64  
dtypes: float64(3), int64(5), object(3)
memory usage: 10.7+ MB


### Check wether same ID present in train and test

In [7]:
set(train_df.id.unique()).intersection(set(test_df.id.unique()))

set()

### Distributions of each attributes in train and test

#### Gender

In [8]:
train_df.Gender.value_counts()

Male      206089
Female    175020
Name: Gender, dtype: int64

In [9]:
test_df.Gender.value_counts()

Male      68236
Female    58801
Name: Gender, dtype: int64

#### Age

In [10]:
train_df.Age.value_counts()

24    25960
23    24256
22    20964
25    20636
21    16457
      ...  
81       56
82       29
83       22
84       11
85       11
Name: Age, Length: 66, dtype: int64

In [11]:
test_df.Age.value_counts()

24    8605
23    8097
22    7094
25    6942
21    5453
      ... 
81      21
82      13
83      10
84       7
85       1
Name: Age, Length: 66, dtype: int64

#### Vehicle Age

In [12]:
train_df.Vehicle_Age.value_counts()

1-2 Year     200316
< 1 Year     164786
> 2 Years     16007
Name: Vehicle_Age, dtype: int64

In [13]:
test_df.Vehicle_Age.value_counts()

1-2 Year     66699
< 1 Year     55019
> 2 Years     5319
Name: Vehicle_Age, dtype: int64

#### Previously Insured

In [14]:
train_df.Previously_Insured.value_counts()

0    206481
1    174628
Name: Previously_Insured, dtype: int64

In [15]:
test_df.Previously_Insured.value_counts()

0    68595
1    58442
Name: Previously_Insured, dtype: int64

#### Annual Premium

In [16]:
train_df.Annual_Premium.value_counts()

2630.0     64877
69856.0      140
39008.0       41
38287.0       38
45179.0       38
           ...  
62326.0        1
59733.0        1
55934.0        1
75387.0        1
53346.0        1
Name: Annual_Premium, Length: 48838, dtype: int64

In [17]:
test_df.Annual_Premium.value_counts()

2630.0     21611
69856.0       45
38452.0       22
45179.0       19
41196.0       17
           ...  
47378.0        1
92601.0        1
43460.0        1
60598.0        1
56336.0        1
Name: Annual_Premium, Length: 36327, dtype: int64

#### Vehicle Damage

In [18]:
train_df.Vehicle_Damage.value_counts()

Yes    192413
No     188696
Name: Vehicle_Damage, dtype: int64

In [19]:
test_df.Vehicle_Damage.value_counts()

Yes    63835
No     63202
Name: Vehicle_Damage, dtype: int64

In [20]:
train_df.Vehicle_Damage.value_counts()

Yes    192413
No     188696
Name: Vehicle_Damage, dtype: int64

In [21]:
test_df.Vehicle_Damage.value_counts()

Yes    63835
No     63202
Name: Vehicle_Damage, dtype: int64

### Append train and test

In [22]:
target = train_df[['Response']]

In [24]:
all_df = train_df.drop(columns='Response').append(test_df)

In [25]:
(all_df[:381109].id==train_df.drop(columns='Response').id).value_counts()

True    381109
Name: id, dtype: int64

In [26]:
(all_df[381109:].id==test_df.reset_index().id).value_counts()

True    127037
Name: id, dtype: int64

In [27]:
all_df

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39
...,...,...,...,...,...,...,...,...,...,...,...
127032,508142,Female,26,1,37.0,1,< 1 Year,No,30867.0,152.0,56
127033,508143,Female,38,1,28.0,0,1-2 Year,Yes,28700.0,122.0,165
127034,508144,Male,21,1,46.0,1,< 1 Year,No,29802.0,152.0,74
127035,508145,Male,71,1,28.0,1,1-2 Year,No,62875.0,26.0,265


In [29]:
all_df['Gender'] = all_df.Gender.apply(lambda x: '0' if x=='Male' else 1)

In [33]:
all_df['Gender'] = all_df['Gender'].astype(int)

In [40]:
all_df['Vehicle_Damage'] = all_df.Vehicle_Damage.apply(lambda x: '0' if x=='No' else 1).astype(int)

In [41]:
all_df['Vehicle_Damage'].value_counts()

1    256248
0    251898
Name: Vehicle_Damage, dtype: int64

In [42]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 508146 entries, 0 to 127036
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    508146 non-null  int64  
 1   Gender                508146 non-null  int64  
 2   Age                   508146 non-null  int64  
 3   Driving_License       508146 non-null  int64  
 4   Region_Code           508146 non-null  float64
 5   Previously_Insured    508146 non-null  int64  
 6   Vehicle_Age           508146 non-null  object 
 7   Vehicle_Damage        508146 non-null  int64  
 8   Annual_Premium        508146 non-null  float64
 9   Policy_Sales_Channel  508146 non-null  float64
 10  Vintage               508146 non-null  int64  
dtypes: float64(3), int64(7), object(1)
memory usage: 46.5+ MB


In [35]:
all_df.Vehicle_Age.value_counts()

1-2 Year     267015
< 1 Year     219805
> 2 Years     21326
Name: Vehicle_Age, dtype: int64

In [44]:
all_df = pd.concat([all_df, pd.get_dummies(all_df.Vehicle_Age)], axis=1)

In [46]:
all_df.drop(['Vehicle_Age','id'], axis=1).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 508146 entries, 0 to 127036
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Gender                508146 non-null  int64  
 1   Age                   508146 non-null  int64  
 2   Driving_License       508146 non-null  int64  
 3   Region_Code           508146 non-null  float64
 4   Previously_Insured    508146 non-null  int64  
 5   Vehicle_Damage        508146 non-null  int64  
 6   Annual_Premium        508146 non-null  float64
 7   Policy_Sales_Channel  508146 non-null  float64
 8   Vintage               508146 non-null  int64  
 9   1-2 Year              508146 non-null  uint8  
 10  < 1 Year              508146 non-null  uint8  
 11  > 2 Years             508146 non-null  uint8  
dtypes: float64(3), int64(6), uint8(3)
memory usage: 40.2 MB


In [47]:
all_df_final  = all_df.drop(['Vehicle_Age','id'], axis=1)

In [49]:
print(train_df.shape)
print(test_df.shape)

(381109, 12)
(127037, 11)


### Models and Submissions!! Yayyy

In [72]:
train = all_df_final[:381109]
test = all_df_final[381109:].reset_index(drop=True)

In [90]:
def make_subs(model, file_name_prefix):
    res = pd.concat([test_df[['id']] , 
                     pd.DataFrame({'Response' : model.predict_proba(test)[:, 1]})], axis=1)
    res.to_csv('submissions/' + file_name_prefix + '_submission.csv', index=False, float_format='%.7f')

In [57]:
import lightgbm as lgbm
import xgboost
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold

from sklearn.model_selection import train_test_split


In [62]:
kfold = KFold(n_splits=10, shuffle=True)

In [55]:
lgbmc = lgbm.LGBMClassifier()

In [68]:
bgc = BaggingClassifier()
rfc = RandomForestClassifier()
xgbc = xgboost.XGBClassifier()

In [64]:
skfold = StratifiedKFold(n_splits=10, shuffle=True)

In [74]:
results = cross_val_score(lgbmc, train, target, cv=skfold, verbose=2)#, scoring='neg_log_loss')
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
results

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV]  ................................................................
[CV] ................................................. , total=   1.6s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s


[CV] ................................................. , total=   1.5s
[CV]  ................................................................
[CV] ................................................. , total=   1.9s
[CV]  ................................................................
[CV] ................................................. , total=   1.5s
[CV]  ................................................................
[CV] ................................................. , total=   1.7s
[CV]  ................................................................
[CV] ................................................. , total=   1.8s
[CV]  ................................................................
[CV] ................................................. , total=   2.1s
[CV]  ................................................................
[CV] ................................................. , total=   2.0s
[CV]  ................................................................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   18.0s finished


array([0.87725329, 0.87735824, 0.87741072, 0.87722705, 0.87712209,
       0.87683346, 0.87712209, 0.8769909 , 0.877332  , 0.87656783])

In [75]:
lgbmc.fit(train, target)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [87]:
target.value_counts()

Response
0           334399
1            46710
dtype: int64

In [86]:
lgbmc.predict_proba(test)[:, 0]

array([1.        , 0.68442213, 0.70448201, ..., 0.99998835, 0.99997598,
       0.99987923])

In [91]:
make_subs(lgbmc, 'lgbc_first')

In [147]:
lgbmc = lgbm.LGBMClassifier(n_estimators=55, num_leaves=50, subsample_for_bin=50000, learning_rate=0.085)

In [148]:
results = cross_val_score(lgbmc, train, target, cv=skfold, verbose=2)#, scoring='neg_log_loss')
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
results

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV]  ................................................................
[CV] ................................................. , total=   1.1s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s


[CV] ................................................. , total=   1.0s
[CV]  ................................................................
[CV] ................................................. , total=   1.0s
[CV]  ................................................................
[CV] ................................................. , total=   1.0s
[CV]  ................................................................
[CV] ................................................. , total=   1.0s
[CV]  ................................................................
[CV] ................................................. , total=   1.1s
[CV]  ................................................................
[CV] ................................................. , total=   1.2s
[CV]  ................................................................
[CV] ................................................. , total=   1.1s
[CV]  ................................................................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   11.3s finished


array([0.87730576, 0.87748944, 0.87785679, 0.87762063, 0.87783055,
       0.87764687, 0.87756816, 0.8774632 , 0.87743696, 0.8775387 ])

In [149]:
# [0.87725329, 0.87735824, 0.87741072, 0.87722705, 0.87712209,
# 0.87683346, 0.87712209, 0.8769909 , 0.877332  , 0.87656783]

# 55, child=50: [0.87748944, 0.87735824, 0.87764687, 0.87748944, 0.87704337,
#        0.87772559, 0.87756816, 0.87772559, 0.87741072, 0.87772238]

In [150]:
lgbmc.fit(train, target)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.085, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=55, n_jobs=-1, num_leaves=50, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=50000, subsample_freq=0)

In [151]:
make_subs(lgbmc, 'lgbc_fourth')

In [119]:
etc = ExtraTreesClassifier()
abc = AdaBoostClassifier()
gbc = GradientBoostingClassifier()
rfc = RandomForestClassifier()


In [120]:
results = cross_val_score(etc, train, target, cv=skfold, verbose=2)#, scoring='neg_log_loss')
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
results

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  ................................................................
[CV] ................................................. , total=   2.8s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.0s remaining:    0.0s
/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV] ................................................. , total=   2.9s
[CV]  ................................................................


/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


...

[CV] ................................................. , total=   3.4s
[CV]  ................................................................


/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV] ................................................. , total=   3.7s
Baseline: 85.96% (0.11%)


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   32.2s finished


array([0.86011913, 0.85843982, 0.86006665, 0.86053895, 0.85838734,
       0.85791504, 0.85933195, 0.85938443, 0.86166724, 0.86011546])

In [127]:
abc = AdaBoostClassifier()

In [128]:
results = cross_val_score(abc, train, target, cv=skfold, verbose=2)#, scoring='neg_log_loss')
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
results

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV]  ................................................................
[CV] ................................................. , total=  11.8s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.0s remaining:    0.0s
/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] ................................................. , total=  11.9s
[CV]  ................................................................


/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

[CV] ................................................. , total=  11.9s
[CV]  ................................................................


/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] ................................................. , total=  11.1s
Baseline: 87.74% (0.00%)


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  1.9min finished


array([0.87743696, 0.87743696, 0.87743696, 0.8774632 , 0.87743696,
       0.87735824, 0.87741072, 0.87741072, 0.87743696, 0.87743374])

In [137]:
abc.fit(train, target)
make_subs(abc, 'abc_first')

/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [133]:
gbc = GradientBoostingClassifier(n_estimators=50)

In [134]:
results = cross_val_score(gbc, train, target, cv=skfold, verbose=2)#, scoring='neg_log_loss')
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
results

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV]  ................................................................
[CV] ................................................. , total=  12.0s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.1s remaining:    0.0s
/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] ................................................. , total=  11.8s
[CV]  ................................................................


/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

[CV] ................................................. , total=  13.6s
[CV]  ................................................................


/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] ................................................. , total=  11.9s
Baseline: 87.74% (0.00%)


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  2.1min finished


array([0.87743696, 0.87743696, 0.87743696, 0.87743696, 0.87743696,
       0.87743696, 0.87743696, 0.87743696, 0.87743696, 0.87743374])

In [138]:
gbc.fit(train, target)
make_subs(gbc, 'gbc_first')

/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [69]:
results = cross_val_score(bgc, train, target, cv=skfold, verbose=2)#, scoring='neg_log_loss')
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
results

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/ensemble/bagging.py:623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV]  ................................................................
[CV] ................................................. , total=  14.0s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   14.2s remaining:    0.0s
/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/ensemble/bagging.py:623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] ................................................. , total=  14.0s
[CV]  ................................................................


/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/ensemble/bagging.py:623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

[CV] ................................................. , total=  14.0s
[CV]  ................................................................


/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/ensemble/bagging.py:623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] ................................................. , total=  14.0s
Baseline: 86.13% (0.09%)


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  2.3min finished


array([0.85977802, 0.86074887, 0.86006665, 0.86287424, 0.86190339,
       0.86122117, 0.86169347, 0.86219202, 0.86169347, 0.86111257])

In [65]:
results = cross_val_score(lgbmc, train, target, cv=skfold, verbose=2)#, scoring='neg_log_loss')
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
results

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV]  ................................................................
[CV] ................................................. , total=   1.5s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s


[CV] ................................................. , total=   1.4s
[CV]  ................................................................
[CV] ................................................. , total=   1.9s
[CV]  ................................................................
[CV] ................................................. , total=   1.6s
[CV]  ................................................................
[CV] ................................................. , total=   1.8s
[CV]  ................................................................
[CV] ................................................. , total=   2.3s
[CV]  ................................................................
[CV] ................................................. , total=   1.8s
[CV]  ................................................................
[CV] ................................................. , total=   1.6s
[CV]  ................................................................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   17.5s finished


array([0.87738448, 0.877332  , 0.87727953, 0.87743696, 0.87717457,
       0.877332  , 0.87725329, 0.8775944 , 0.87738448, 0.87709263])

In [67]:
results = cross_val_score(rfc, train, target, cv=skfold, verbose=2)#, scoring='neg_log_loss')
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
results

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  ................................................................
[CV] ................................................. , total=   5.3s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.4s remaining:    0.0s
/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV] ................................................. , total=   5.1s
[CV]  ................................................................


/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


...

[CV] ................................................. , total=   5.3s
[CV]  ................................................................


/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV] ................................................. , total=   5.3s
Baseline: 86.29% (0.03%)


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   51.4s finished


array([0.86287424, 0.862848  , 0.86276928, 0.86263808, 0.86316287,
       0.86250689, 0.86353021, 0.86303167, 0.86311039, 0.86292312])